In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

In [1]:
import weaviate
from weaviate.embedded import EmbeddedOptions

# client = weaviate.connect_to_embedded()

weaviate_client = weaviate.WeaviateClient(
    embedded_options=EmbeddedOptions(
        persistence_data_path="../weaviate_persistence"
    )
)

weaviate_client

{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-10-10T22:00:09Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-10-10T22:00:09Z"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-10-10T22:00:09Z"}
{"level":"info","msg":"module offload-s3 is enabled","time":"2024-10-10T22:00:09Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-10-10T22:00:09Z"}
{"level":"info","msg":"open cluster service","servers":{"Embedded_at_8079":36507},"time":"2024-10-10T22:00:09Z"}
{"address":"172.17.0.2:36

In [ ]:
from unstructured.chunking.title import chunk_by_title
from langchain_core.documents import Document
from unstructured.partition.auto import partition

def get_langchain_docs(elements, filename):

    # create chunks to better rag
    chunks = chunk_by_title(elements, max_characters=1500, new_after_n_chars=1000, overlap=150)
    
    documents = []
    # convert to the Langchain format
    for chunk in chunks:
        metadata = chunk.metadata.to_dict()
        # print("---------------------")
        # print(metadata)
        # print("---------------------")
        # print(chunk.text)

        del metadata["languages"]
        if "filename" in metadata:
            metadata["source"] = metadata["filename"]
        else:
            metadata["source"] = filename
            metadata["filename"] = filename

        documents.append(Document(page_content=chunk.text, metadata=metadata))

    return documents

In [ ]:
folder_path = "../docs"

path, dirs, files = next(os.walk(folder_path))
        
documents_all = []
for file in files:
    with open( os.path.join(path, file), 'rb') as f:
        print(f'Opening file: {file}')
        
        # open file, clean, segment
        elements = partition(file=f)
        
        documents = get_langchain_docs(elements, file)
        documents_all.extend(documents)

documents_all

In [ ]:
print( documents_all[0].page_content )

In [4]:
from langchain_community.embeddings import GPT4AllEmbeddings
        
model_name = "nomic-embed-text-v1.f16.gguf"

gpt4all_kwargs = {'allow_download': 'True'}

embeddings_provider = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)
embeddings_provider

/workspaces/llm-rag-evaluation-ragas/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


GPT4AllEmbeddings(model_name='nomic-embed-text-v1.f16.gguf', n_threads=None, device='cpu', gpt4all_kwargs={'allow_download': 'True'}, client=<gpt4all.gpt4all.Embed4All object at 0x7f75cc42ab50>)

In [7]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

index_name = "index1"

In [ ]:
weaviate_client.connect()

vector_store = WeaviateVectorStore.from_documents(documents,  embeddings_provider, client=weaviate_client, index_name=index_name)
vector_store

In [9]:
weaviate_client.connect()
vector_store = WeaviateVectorStore(weaviate_client, index_name, "text", embedding=embeddings_provider)

# retriever = vector_store.as_retriever(search_type="mmr", fetch_k=20)
retriever = vector_store.as_retriever(search_type="similarity", k=4)  #default settings from langchain

In [10]:
retriever.invoke("pje")

[Document(metadata={'filetype': 'application/pdf', 'filename': 'PJe Mídias Desktop - Envio de Perícias em MP4.pdf', 'page_number': 4.0, 'orig_elements': 'eJzlWl1z2zYW/SsYPbsKvj/65jru1pkm8djuvqSdDD4ubHYlUiUp12ln//tekHIiy5rY3lSupnnx8IAQCd6De3EO4Hd/TmAGc6j791WafEsm2UnBMkCkJupMjQ2cO8FEDtapyOXkgEzm0Pvke4/9/5zEpmlTVfseugHP/Idm2b+/guryqscWKzn+ZNX6e5X6K2xUTmPjoqnqvvzq3Tuh2BSbjDFT+csB+YitHbHiYiq24KE/Nky6D10P8/IFp9UNzM4XPsLkv3gjVzPoPyyg3PKLxayKvq+a+sUi5ckwsPpy6S+Hwb+bQH05KU9bYMv7ejkP0GK7HFraT0FiwWvJnfVScmYCsKhyMJpJCkEp5ocX93BTvn/yGkPTVn5GYOZD0/rUkAVekvPji5OjF0cn52UYtyO8qPrZMO5NXkJM0URBs3Ypei9wBEwwzcEEJRnwHfGimClxttRN2RD3FeZshUce7uGh/855WQ+zWA/j902DQd8WR89EAsFF5DGKzA134JxyFJjMQhq1ozgyyafqgLDV9F5BY6e6QMno1N2D7DnmtlqP4enbl8dn5NVPL0+OTn5eUhrZ2clbcnF28t1Pbw5/JGfH/zp5Wy5elsbD7w5//OEtOYfYYrxwhpME5AJi3cyayxGd1Llp5748CszwVzQEyFEzX9Y41rvtRyXQUGN+rJ51Ml/gF/Qb3fDGeVXi4bt1xn8An7Yzzr2l0YGPkkP2LkvMGGE1VTaCDdnvinFqpxRJpHrqBspXmNkRK86nZgse+u+c9c2K9mBarE+TI981xJP5QAn7bYmxIr+OgCyarlt6spx7cg1t95G1hpy+gtcDTKnyHRkuHa

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_rag_basic = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "Question: {question} "
    "Context: {context} "
    "Answer: "
)

prompt_rag_basic_lc = PromptTemplate.from_template(prompt_rag_basic)

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=500,
)

In [13]:
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

context_docs_used = None

def format_docs(docs):
    # print("Documents retrieved used as context:")
    # for doc in docs:
    #     print(f"=============================== Filename: {doc.metadata['filename']}")
    #     print(doc.page_content)

    global context_docs_used
    context_docs_used = docs

    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_1_basic = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_rag_basic_lc
    | llm_gemini
    | StrOutputParser()
)

In [14]:
response = rag_chain_1_basic.invoke("como acessar o PJe?")

print("================== AI response")
print( response )

================== AI response
To access PJeMídias Desktop, you need to enter your CPF in the username field and your registered password. Once you log in, the system will direct you to the main screen.  You can then access the "NovaPerícia / Prova" option from the menu at the top left of the screen. 



In [19]:
from langchain.schema import AIMessage, HumanMessage

def gr_rag_chain_1_basic(message, history):
    response = rag_chain_1_basic.invoke(message)
    
    return response

In [20]:
import gradio as gr

chat_ui = gr.ChatInterface(gr_rag_chain_1_basic, type="messages")

chat_ui.launch(share=True)

/workspaces/llm-rag-evaluation-ragas/.venv/lib/python3.11/site-packages/gradio/http_server.py:119: ResourceWarning: unclosed <socket.socket fd=89, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/workspaces/llm-rag-evaluation-ragas/.venv/lib/python3.11/site-packages/gradio/http_server.py:119: ResourceWarning: unclosed <socket.socket fd=90, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://4eb6abf01c65894bc8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/workspaces/llm-rag-evaluation-ragas/.venv/lib/python3.11/site-packages/starlette/templating.py:161: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(


In [ ]:
from langchain.schema import AIMessage, HumanMessage

def gr_rag_chain_1_basic(message, history):
    history_langchain_format = []

    for msg in history:
        if msg['role'] == "user":
            history_langchain_format.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_langchain_format.append(AIMessage(content=msg['content']))
    
    history_langchain_format.append(HumanMessage(content=message))
    
    gpt_response = rag_chain_1_basic.invoke(history_langchain_format)
    
    return gpt_response.content